### 데이터셋 다운로드 
- 데이터 URL 사용
    - urllib 패키지 활용 : from urllib.request import urlretreive
    - 함수 : 

In [35]:
### 모듈로딩
from urllib.request import urlretrieve
import os

### 파이토치의 내장 이미지 데이터셋
from torchvision import datasets

In [36]:
LOGO_url='https://www.google.com/images/branding/googlelogo/2x/googlelogo_color_272x92dp.png'
PATH='../image/'
FILE_NAME='../image/logo.png'

In [37]:
# 폴더 존재 여부 확인 후 생성
if not os.path.exists(PATH):
    os.mkdir(PATH)

In [38]:
# 데이터 다운로드 후 이미지 저장
urlretrieve(LOGO_url,FILE_NAME) 

('../image/logo.png', <http.client.HTTPMessage at 0x19754dcf8e0>)

In [39]:
### MNIST 데이터셋 다운로드
datasets.MNIST('../data/',download=True)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ../data/
    Split: Train

- 바이너리 데이터 읽기

In [40]:
# 데이터 파일리스트 준비
MNIST_path='../data/MNIST/raw/'
filelist=os.listdir(MNIST_path)

print(filelist)

for file in filelist:
    if file.endswith('.gz'):
        filelist.remove(file)

print(filelist)


['t10k-images-idx3-ubyte', 't10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte', 't10k-labels-idx1-ubyte.gz', 'train-images-idx3-ubyte', 'train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte', 'train-labels-idx1-ubyte.gz']
['t10k-images-idx3-ubyte', 't10k-labels-idx1-ubyte', 'train-images-idx3-ubyte', 'train-labels-idx1-ubyte']


In [41]:
# 데이터 파일 읽기
with open(MNIST_path+filelist[0],mode='rb') as f:
    data=f.read(4)     # 4 바이트만 읽어줘!
    print(data)

    data=f.read(4)     # 4 바이트만 읽어줘!
    print(data)

b'\x00\x00\x08\x03'
b"\x00\x00'\x10"


- 바이너리 데이터를 원하는 자료형으로 읽기
    - 파이썬 내장 모듈 struct 사용

In [42]:
# 모듈 로딩
import struct

In [82]:
with open(MNIST_path+filelist[0],mode='rb') as f:
    _,img_cnt=struct.unpack('>II',f.read(8))       # 매직넘버, 사진갯수
    rows,cols=struct.unpack('>II',f.read(8)) 
    
    # 위의 정보는 우리가 봐야하므로 언팩킹
    # 밑의 정보는 컴퓨터가 읽을 정보이므로 굳이 언팩킹 xx
    pixels=f.read(rows*cols)
    print(pixels)
    print(len(pixels))
    print(list(map(lambda n:str(n),pixels)))

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00T\xb9\x9f\x97<$\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xde\xfe\xfe\xfe\xfe\xf1\xc6\xc6\xc6\xc6\xc6\xc6\xc6\xc6\xaa4\x00\x00\x00\x00\x00\x00\x

In [72]:
# 데이터 파일 읽어서 데이터셋 파일 생성
def makeDataFile(imgfile,labelfile,datasetfile):
    with open(imgfile,mode='rb') as data_f:
        with open(labelfile,mode='rb') as label_f:
            with open(datasetfile,mode='w') as dataset_f:
                # strucct.unpack('>I',f.read(4))       # 4  바이트만 변환할때
                # 파일의 헤더정보 읽기
                _,img_cnt=struct.unpack('>II',data_f.read(8))       # 매직넘버, 사진갯수
                rows,cols=struct.unpack('>II',data_f.read(8))       # 사진한장의 로우와 컬럼
                
                _,_=struct.unpack('>II',label_f.read(8))            # 라벨에서는 쓸모없는 데이터라 버림!
            
                for n in range(img_cnt):
                    # 이미지 raw데이터
                    pixels=data_f.read(rows*cols)
                    sdata=list(map(lambda n:str(n),pixels))
                    #print(sdata)

                    # 이미지 라벨
                    label=struct.unpack('B',label_f.read(1))[0]

                    # 피쳐와 라벨 연결
                    sdata.append(str(label))
                    #print(sdata)

                    img_label= ','.join(sdata)
                    dataset_f.write(img_label+'\n')

                    #print(','.join(sdata))

In [83]:
DATASET_PATH='../data/'
TRAIN_FILE=DATASET_PATH+'mnist_train.csv'
TEST_FILE=DATASET_PATH+'mnist_test.csv'
makeDataFile(MNIST_path+filelist[0],MNIST_path+filelist[1],TEST_FILE)
makeDataFile(MNIST_path+filelist[2],MNIST_path+filelist[3],TRAIN_FILE)